# Twitter Analysis Pipeline

![](https://www.evernote.com/l/AAFtiWq4lUNNiqyswF72dUV3f_7Geq_TcykB/image.png)

In [1]:
! pip install boto3 pymongo twitter --quiet

In [2]:
import lib 

## Write Tweets to S3

### Create a Tweet Iterator

In [3]:
consumer_key    = 'LjPot0s4ipAJ1R2UzcT91WWXs'
consumer_secret = '7xwm9jjV8w5yrepi4K24ozskeXlXn1QNHz7MvxAjcIza0vIsrs'
token           = '1020138399552790528-NSoD9LgEkA1ne2oBDFogOvfJkE2r66'
token_secret    = 'GsakoOIlM1HJ58uU4HqK4WnqxWFgiWMyMHhoAhjfwWrCz'
bounding_box    = "-118.5137323688,34.0001996344,-118.4702449172,34.0331651696"

tweet_iterator  = lib.create_tweet_iterator(token, 
                                            token_secret,
                                            consumer_key,
                                            consumer_secret,
                                            bounding_box)

In [4]:
tweets = [next(tweet_iterator) for _ in range(50)]

In [5]:
tweets_2 = []
for _ in range(50):
    tweets_2.append(next(tweet_iterator))

### Write list of tweets to JSON file on disk

In [6]:
import json 

username = 'mark'

filename = lib.create_timestamped_filename(username)
with open(filename, 'w') as outfile:
    json.dump(tweets, outfile)
    
filename = lib.create_timestamped_filename(username)
with open(filename, 'w') as outfile:
    json.dump(tweets_2, outfile)    

### Write Tweet Files to S3, Step 1 - Create a Boto Client to S3

https://boto3.readthedocs.io

In [7]:
s3_client = lib.create_boto_client()

In [8]:
from os import listdir

In [10]:
current_directory = listdir()
current_directory = [file for file in current_directory if 'tweets-mark' in file]
current_directory

['tweets-mark-2018-07-26_01-15-10-970777.json',
 'tweets-mark-2018-07-24_03-51-56-187383.json',
 'tweets-mark-2018-07-26_01-15-10-950495.json',
 'tweets-mark-2018-07-24_02-26-34-669341.json',
 'tweets-mark-2018-07-24_02-34-15-562280.json',
 'tweets-mark-2018-07-24_03-59-20-478995.json',
 'tweets-mark-2018-07-24_02-25-50-384070.json',
 'tweets-mark-2018-07-24_02-37-46-350548.json',
 'tweets-mark-2018-07-24_02-29-27-268409.json',
 'tweets-mark-2018-07-24_03-57-55-937055.json']

In [11]:
from os import rename

### Write Tweet Files to S3, Step 2 - Write a file to S3

In [12]:
S3_BUCKET = 'uclax-data-science'

for filename in current_directory:
    lib.write_file_to_S3(s3_client, filename, S3_BUCKET)
    old_filename = filename
    filename = filename.replace('.json', '')
    filename = filename + '-processed.json'
    rename(old_filename, filename)

In [15]:
current_directory = listdir()
current_directory = [file for file in current_directory if 'tweets-mark' in file]
current_directory

['tweets-mark-2018-07-24_02-29-27-268409-processed.json',
 'tweets-mark-2018-07-24_02-25-50-384070-processed.json',
 'tweets-mark-2018-07-26_01-15-10-950495-processed.json',
 'tweets-mark-2018-07-24_03-51-56-187383-processed.json',
 'tweets-mark-2018-07-26_01-15-10-970777-processed.json',
 'tweets-mark-2018-07-24_02-37-46-350548-processed.json',
 'tweets-mark-2018-07-24_03-57-55-937055-processed.json',
 'tweets-mark-2018-07-24_02-34-15-562280-processed.json',
 'tweets-mark-2018-07-24_03-59-20-478995-processed.json',
 'tweets-mark-2018-07-24_02-26-34-669341-processed.json']

### List Files on S3

In [16]:
s3_files = lib.list_files_in_S3_bucket(s3_client, S3_BUCKET)
s3_files = [file for file in s3_files if 'tweets-mark' in file]
s3_files

['tweets-mark-2018-07-24_02-25-50-384070.json',
 'tweets-mark-2018-07-24_02-26-34-669341.json',
 'tweets-mark-2018-07-24_02-29-27-268409.json',
 'tweets-mark-2018-07-24_02-34-15-562280.json',
 'tweets-mark-2018-07-24_02-37-46-350548.json',
 'tweets-mark-2018-07-24_03-51-56-187383.json',
 'tweets-mark-2018-07-24_03-57-55-937055.json',
 'tweets-mark-2018-07-24_03-59-20-478995.json',
 'tweets-mark-2018-07-26_01-15-10-950495.json',
 'tweets-mark-2018-07-26_01-15-10-970777.json']

## Write Tweets to Mongo

### Read an object from S3

In [21]:
S3_BUCKET = 'uclax-data-science'
key = s3_files[1]

tweets_from_s3 = lib.read_object_from_S3(s3_client, 'tweets-mark-2018-07-26_01-15-10-970777.json', S3_BUCKET)

### Write Tweets to Mongo

In [22]:
from pymongo.errors import DuplicateKeyError

In [23]:
collection_client = lib.create_mongo_client_to_database_collection('twitter', 'tweets')
for tweet in tweets_from_s3:
    try:
        collection_client.insert_one(tweet)
    except DuplicateKeyError as e:
        print(e)

In [24]:
collection_client = lib.create_mongo_client_to_database_collection('twitter', 'tweets')
for tweet in tweets_from_s3:
    try:
        collection_client.insert_one(tweet)
    except DuplicateKeyError as e:
        print(e)

E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227539cb64017fc81e15') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227639cb64017fc81e16') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227639cb64017fc81e17') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227639cb64017fc81e18') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227639cb64017fc81e19') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227739cb64017fc81e1a') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227739cb64017fc81e1b') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227739cb64017fc81e1c') }
E11000 duplicate key error collection: twitter.tweets in

In [25]:
for key in s3_files[2:]:
    for tweet in tweets_from_s3:
        try:
            collection_client.insert_one(tweet)
        except DuplicateKeyError as e:
            print(e)

E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227539cb64017fc81e15') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227639cb64017fc81e16') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227639cb64017fc81e17') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227639cb64017fc81e18') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227639cb64017fc81e19') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227739cb64017fc81e1a') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227739cb64017fc81e1b') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227739cb64017fc81e1c') }
E11000 duplicate key error collection: twitter.tweets in

E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227839cb64017fc81e29') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227839cb64017fc81e2a') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227839cb64017fc81e2b') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227839cb64017fc81e2c') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227839cb64017fc81e2d') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227939cb64017fc81e2e') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227939cb64017fc81e2f') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227939cb64017fc81e30') }
E11000 duplicate key error collection: twitter.tweets in

E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227a39cb64017fc81e3e') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227a39cb64017fc81e3f') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227a39cb64017fc81e40') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227a39cb64017fc81e41') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227a39cb64017fc81e42') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227b39cb64017fc81e43') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227b39cb64017fc81e44') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227b39cb64017fc81e45') }
E11000 duplicate key error collection: twitter.tweets in

E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227739cb64017fc81e21') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227739cb64017fc81e22') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227739cb64017fc81e23') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227739cb64017fc81e24') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227739cb64017fc81e25') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227839cb64017fc81e26') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227839cb64017fc81e27') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227839cb64017fc81e28') }
E11000 duplicate key error collection: twitter.tweets in

E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227939cb64017fc81e36') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227939cb64017fc81e37') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227939cb64017fc81e38') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227a39cb64017fc81e39') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227a39cb64017fc81e3a') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227a39cb64017fc81e3b') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227a39cb64017fc81e3c') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227a39cb64017fc81e3d') }
E11000 duplicate key error collection: twitter.tweets in

E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227639cb64017fc81e19') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227739cb64017fc81e1a') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227739cb64017fc81e1b') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227739cb64017fc81e1c') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227739cb64017fc81e1d') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227739cb64017fc81e1e') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227739cb64017fc81e1f') }
E11000 duplicate key error collection: twitter.tweets index: _id_ dup key: { : ObjectId('5b59227739cb64017fc81e20') }
E11000 duplicate key error collection: twitter.tweets in

## One More Time

In [26]:
tweets = [next(tweet_iterator) for _ in range(50)]

filename = lib.create_timestamped_filename(username)
with open(filename, 'w') as outfile:
    json.dump(tweets, outfile)

In [28]:
current_directory = listdir()
current_directory = [file for file in current_directory 
                     if ('tweets-mark' in file) and ('processed' not in file)]
current_directory

['tweets-mark-2018-07-26_01-25-27-443503.json']

In [29]:
for filename in current_directory:
    lib.write_file_to_S3(s3_client, filename, S3_BUCKET)
    old_filename = filename
    filename = filename.replace('.json', '')
    filename = filename + '-processed.json'
    rename(old_filename, filename)

In [30]:
s3_files = lib.list_files_in_S3_bucket(s3_client, S3_BUCKET)
s3_files = [file for file in s3_files if 'tweets-mark' in file]
s3_files

['tweets-mark-2018-07-24_02-25-50-384070.json',
 'tweets-mark-2018-07-24_02-26-34-669341.json',
 'tweets-mark-2018-07-24_02-29-27-268409.json',
 'tweets-mark-2018-07-24_02-34-15-562280.json',
 'tweets-mark-2018-07-24_02-37-46-350548.json',
 'tweets-mark-2018-07-24_03-51-56-187383.json',
 'tweets-mark-2018-07-24_03-57-55-937055.json',
 'tweets-mark-2018-07-24_03-59-20-478995.json',
 'tweets-mark-2018-07-26_01-15-10-950495.json',
 'tweets-mark-2018-07-26_01-15-10-970777.json',
 'tweets-mark-2018-07-26_01-25-27-443503.json']

In [31]:
s3_files[-1]

'tweets-mark-2018-07-26_01-25-27-443503.json'

In [32]:
key = s3_files[-1]

In [33]:
tweets_from_s3 = lib.read_object_from_S3(s3_client, key, S3_BUCKET)

In [34]:
collection_client.insert_many(tweets_from_s3)

## Automate It

In [1]:
from datetime import datetime

In [6]:
import lib

In [7]:
TOKEN           = "12270642-8ggOVEjcQ7DAP2VCCZbXYoRtiOf0rngzrsyb82h7k"
TOKEN_SECRET    = "85KnJJBbdvhZyGhOVLPwk47x2IAHaLEFGhNJdrHXo1Tv8"
CONSUMER_KEY    = "Jt9H3ve228Tl8yafi4Ip8Iwbw"
CONSUMER_SECRET = "xXxkq7QM1D7K1w0tFKRIYYviS2EQ288jzbbiwzEYYRQrMyvCDN"
BOUNDING_BOX    = "-118.5137323688,34.0001996344,-118.4702449172,34.0331651696"

DATABASE        = 'twitter'
COLLECTION      = 'tweets'
USERNAME        = 'mark'

S3_BUCKET       = 'uclax-data-science'

In [8]:
tweet_iterator    = lib.create_tweet_iterator(TOKEN, TOKEN_SECRET, 
                                              CONSUMER_KEY, CONSUMER_SECRET, BOUNDING_BOX)
s3_client         = lib.create_boto_client()
collection_client = lib.create_mongo_client_to_database_collection(DATABASE, COLLECTION)

In [9]:
def timestamp():
    now = datetime.now().strftime('%D %H:%M:%S')
    print(now, end=' | ')
    print('Collecting Tweets', end=' | ')
    
def collect_tweets(n=50):
    tweets = [next(tweet_iterator) for _ in range(n)]
    print('{} Tweets'.format(n), end=' | ')
    return tweets
    
def write_to_disk(tweets):
    filename = lib.create_timestamped_filename(USERNAME) 
    with open(filename, 'w') as outfile:
        json.dump(tweets, outfile)    
    print('Written to Disk', end=' | ')
    return filename
    
def write_to_S3(filename):   
    lib.write_file_to_S3(s3_client, filename, S3_BUCKET)
    old_filename = filename
    filename = filename.replace('.json', '')
    filename = filename + '-processed.json'
    rename(old_filename, filename)
    print('Written to S3', end=' | ')
    
def insert_to_mongo(key):
    tweets_from_s3 = lib.read_object_from_S3(s3_client, key, S3_BUCKET)    
    collection_client.insert_many(tweets_from_s3)
    print('Inserted to Mongo')

In [10]:
while True:
    timestamp()
    tweets = collect_tweets()
    filename = write_to_disk(tweets)
    write_to_S3(filename)
    insert_to_mongo(filename)

07/26/18 01:51:19 | Collecting Tweets | 50 Tweets | 

NameError: name 'json' is not defined